# Cyclic ID Algorithm: High-Level Workflow and Examples

This notebook will demonstrate the complete cyclic ID algorithm for identifying causal effects in cyclic graphs with observational data.


**This notebook will show:**

- Examples using a graph with Identifiable queries ✅
- Examples using a graph with Unidentifiable queries ❌

**Note**: This is a high-level demo showing the full workflow. For more information please see other implementation notebooks and explanation notebooks.

In [ ]:
from y0.dsl import A, B, C, D, E, X, Y
from y0.graph import NxMixedGraph

## Example 1: Identifiable Cyclic Graph

### Graph Structure 

For this high level example we have a minimal example of **unidentifiability** first in a cyclic graph. 

- **X → X**: Self-loop on X (feedback)
- **X → Y**: X influences Y
- **Y → X**: Y influences X

X and Y form a feedback loop, which is also known as a strongly connected component.


### Graph Description

This graph has:

- **R → X**: External input to the cycle
- **X ⇄ Y**: Bidirectional cycle between X and Y
- **X → X**: Self-loop on X
- **X → Z**: Path from cycle to outcome

**Expected Result**: This query is expected to be identifiable. With a real graph, we usually would not know if it is identifiable or not. Various queries might either be identifiable or unidentifiable. 

In [ ]:
# Define Graph 1: Identifiable Cyclic Graph Example with an identifiable query.

graph_1 = NxMixedGraph.from_edges(
    directed=[
        (X, X),
        (X, Y),
        (Y, X),
    ]
)

print("Graph 1 created.")
print(f"Nodes: {list(graph_1.nodes())}")
print(f"Directed edges: {list(graph_1.directed.edges())}")

## Implementing the Cyclic ID Algorithm

Below we will implement the main Cyclic ID algorithm that will determine if the example graphs we have are identifiable or not.

**What the algorithm does:**

The Cyclic ID algorithm solves a fundamental problem: *What causal queries can be identified on a cyclic graph?*

**High-level strategy:**

1. **Find relevant variables**: Identify which variables are actually needed (ancestors of our outcome).
2. **Break into pieces**: Split the graph into manageable "districts", or groups of variables that need to be handled together.
3. **Identify each piece**: Use the IDCD algorithm to identify each district separately.
4. **Combine results**: Multiply the identified pieces together and focus on our outcome variable.

**Why this works for cycles:**

Traditional methods assume acyclic graphs (no feedback loops). This algorithm handles cycles by:
- Treating feedback loops as unified blocks (consolidated districts)
- Using a special ordering (apt-order) instead of topological order
- Recursively breaking down complex structures until each piece is identifiable

### Running Cyclic ID Algorithm on Graph 1

Now below we can call the function on the first example graph to identify the query we have. We can do this same action for several queries in a directed mixed graph. First we see the use of a helper function in order to get the initial distribution.


Now we can make a call to the main cyclic ID algorithm below in order to see if several examples of queries are truly identifiable or 
unidentifiable.

As seen above, the queries for the example minimal graph are all considered identifiable. Below we can create a more complex graph that does have a few identifiable queries for the algorithm to identify. 



In [ ]:
# Create complex cyclic graph
complex_graph = NxMixedGraph.from_edges(
    directed=[
        (A, B),  # External input
        (B, B),  # Self-loop
        (B, C),  # First cycle edge 1
        (C, B),  # First cycle edge 2
        (C, D),  # Connection between cycles
        (D, E),  # Second cycle edge 1
        (E, D),  # Second cycle edge 2
    ]
)